In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
df = pd.read_csv('sredjeni_kurs.csv', parse_dates=True)

In [4]:
from sklearn.model_selection import train_test_split

# Podela podataka na testni i trening skup
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

In [5]:
# Definisanje parametara pretrage
stepwise_model = auto_arima(train_data['MIDDLE EXCHANGE RATE'], 
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            m=12,  # broj vremenskih koraka u jednoj sezoni
                            start_P=0, seasonal=True,
                            d=1, D=1, trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=17.43 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=4514.722, Time=0.37 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2135.298, Time=2.41 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=12.60 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=4168.702, Time=0.85 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=1398.061, Time=5.32 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=41.01 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=22.20 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=1755.000, Time=2.59 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=1369.329, Time=7.03 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=2098.972, Time=3.92 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=43.60 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=inf, Time=30.94 sec
 ARIMA(3,1,0)(2,1,0)[12]             : AIC=1370.849, Time=6.65 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=

In [6]:
# Ispis optimalnih parametara
print("Optimalni parametri ARIMA:", stepwise_model.order)
print("Optimalni sezonski parametri ARIMA:", stepwise_model.seasonal_order)

Optimalni parametri ARIMA: (2, 1, 1)
Optimalni sezonski parametri ARIMA: (2, 1, 0, 12)


In [7]:
# Treniranje SARIMAX modela sa optimalnim parametrima
sarimax_model = SARIMAX(train_data['MIDDLE EXCHANGE RATE'], order=stepwise_model.order,
                        seasonal_order=stepwise_model.seasonal_order)
sarimax_results = sarimax_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.55469D-01    |proj g|=  1.07504D+00


 This problem is unconstrained.



At iterate    5    f=  1.10836D-01    |proj g|=  6.36798D-03

At iterate   10    f=  1.10787D-01    |proj g|=  4.55519D-04

At iterate   15    f=  1.10787D-01    |proj g|=  1.09288D-03

At iterate   20    f=  1.10668D-01    |proj g|=  9.38036D-02

At iterate   25    f=  1.10325D-01    |proj g|=  5.42699D-02

At iterate   30    f=  1.10117D-01    |proj g|=  9.78529D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     34     42      1     0     0   8.349D-06   1.101D-01
  F =  0.11011548604269111     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [8]:
# Predviđanja za testni skup
sarimax_forecast = sarimax_results.predict(start=test_data.index[0], end=test_data.index[-1], typ='levels')

# Izračunavanje metrika
mse = mean_squared_error(test_data['MIDDLE EXCHANGE RATE'], sarimax_forecast)
mae = mean_absolute_error(test_data['MIDDLE EXCHANGE RATE'], sarimax_forecast)
rmse = np.sqrt(mse)

# Ispis rezultata
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

/home/ivana/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


Mean Squared Error (MSE): 2.7098264822628897
Mean Absolute Error (MAE): 1.394207234979689
Root Mean Squared Error (RMSE): 1.6461550602123998
